[View in Colaboratory](https://colab.research.google.com/github/jadevaibhav/UNet-for-biomedical-images/blob/master/unet_paper_architecture.ipynb)

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
!ls

drive  sample_data


In [0]:
import os 
os.chdir('drive/My Drive/brat')

In [4]:
!ls

Brain tumor Segementation.gslides  model.ipynb
Brain Tumor Segmentation.gdoc	   siamese_training.ipynb
data.ipynb			   training.ipynb
data_trial_81.h5		   train-labels.tif
data_trial_dim2_128.h5		   train-volume.tif
data_Y_0001.pickle		   trial_0001_2path_acc.h5
FinalCode.ipynb			   trial_0001_accuracy.h5
HG				   trial_0001_input_cascasde_acc.h5
info1_input.h5			   trial_0001_MFCcas_dim2_128_acc.h5
info_mfc.h5			   trial_MFCcascade_acc.h5
LG				   unet_paper_architecture.ipynb
MFC_cascade_siam.h5		   untitled.ipynb


In [6]:
import keras
from keras import layers
from keras.layers import Input, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, Lambda,Concatenate,Cropping2D
from keras.layers import AveragePooling2D, MaxPooling2D, Dropout, GlobalMaxPooling2D, GlobalAveragePooling2D, Add, Conv2DTranspose
from keras.models import Model
from keras import regularizers
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
from keras.initializers import glorot_normal
#import pydot
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model

Using TensorFlow backend.


In [0]:
def crop(in1,in2):
  len1 = int(in1[1])*2
  wid1 = int(in1[2])*2
  len2 = int(in2[1])
  wid2 = int(in2[2])
  ln = int((len2-len1)/2) 
  wid = int((wid2-wid1)/2)
  return (ln,wid)

In [0]:
def conv_block(X,filters):
  X = Conv2D(filters,(3,3))(X)
  X = BatchNormalization()(X)
  X = Activation('relu')(X)
  X = Conv2D(filters,(3,3))(X)
  X = BatchNormalization()(X)
  X = Activation('relu')(X)
  
  return X

In [0]:
def down_block(X,block_no):
  Xa = conv_block(X,64*2**(block_no - 1))
  X = MaxPooling2D((2,2))(Xa)
  print(X.shape)
  
  return(X,Xa)

In [0]:
def up_block(X,Xa):
  crop_shape = crop(X.shape,Xa.shape)
  ln = int(X.shape[1])
  channels = int(Xa.shape[3])
  X = Conv2DTranspose(channels,(2,2),strides=(2,2))(X)
  Xa = Cropping2D(crop_shape)(Xa)
 
  
  if(Xa.shape[1] != ln*2):
    Xa = MaxPooling2D((2,2),(1,1))(Xa)
  print(X.shape,Xa.shape)
  X = Concatenate(axis=-1)([Xa,X])
  print(X.shape)
  
  return X

In [7]:
  input_shape=(572,572,1)
#def unet(input_shape):
  X_in = Input(input_shape)
  
  X,X1 = down_block(X_in,1)
  
  X,X2 = down_block(X,2)
    
  X,X3 = down_block(X,3)
  
  X,X4 = down_block(X,4)
  
  X = conv_block(X,5)
  
  X = up_block(X,X4)
  
  X = conv_block(X,512)
  
  X = up_block(X,X3)
  
  X = conv_block(X,256) 
  print(X.shape)
  
  X = up_block(X,X2)
  
  X = conv_block(X,128) 
  print(X.shape)
  
  X = up_block(X,X1)
               
  X = conv_block(X,64)
  
  X = Conv2D(1,(1,1))(X)
  X = Activation('softmax')(X)
  print(X.shape)
  
  #X = Conv2DTranspose(1,(2,2),strides=(2,2))(X)
  #X = Conv2D(1,(5,5))(X)
  #print(X.shape)
  M = Model(inputs=X_in,outputs=X)

(?, 284, 284, 64)
(?, 140, 140, 128)
(?, 68, 68, 256)
(?, 32, 32, 512)
(?, ?, ?, 512) (?, 56, 56, 512)
(?, 56, 56, 1024)
(?, ?, ?, 256) (?, 104, 104, 256)
(?, 104, 104, 512)
(?, 100, 100, 256)
(?, ?, ?, 128) (?, 200, 200, 128)
(?, 200, 200, 256)
(?, 196, 196, 128)
(?, ?, ?, 64) (?, 392, 392, 64)
(?, 392, 392, 128)
(?, 388, 388, 1)


In [0]:
from skimage import io
import numpy as np

In [8]:
img = io.imread('train-volume.tif')
x = np.asarray(img)
x.shape

(30, 512, 512)

In [9]:
img = io.imread('train-labels.tif')
Y = np.asarray(img)
Y.shape

(30, 512, 512)

In [10]:
Y = Y[:,94:418,94:418]/255
Y.shape

(30, 324, 324)

In [0]:
x = x.reshape((30,512,512,1))
Y = Y.reshape((30,324,324,1))

In [0]:
M.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

**unbalanced classes, stuck into local minima(Performance is better than all 1)**

In [0]:
M.fit(x,y,epochs=2,batch_size=2)

**Trying to resolve unbalanced class**

In [13]:
sm = np.sum(Y)/(30*324*324)
weight = [sm,1-sm]
weight

[0.7659684118274653, 0.23403158817253467]

In [0]:
M.fit(x,Y,epochs=10,class_weight = weight)

Epoch 1/10
